In [21]:
#runs each of the 3 models (1D, two 2D) on the data from the gunrange
    # outputs metrics for each model individually and for the two ensemble methods of the 2D
    # uses folders of the wav files that we individually separated, gunshot and nongunshot (not numpy arrays)
        
    
import pyaudio
import librosa
import wave
import numpy as np
from threading import Thread
from array import array
from scipy.io import wavfile
import soundfile as sf
import scipy.signal
from queue import Queue
import time
import os
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import cv2
import six
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Input, layers, optimizers, backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelBinarizer


## Variables

In [2]:
#variables
audio_sample_duration = 2
sound_data = np.zeros(0, dtype = "int16")
sound_normalization_threshold = 10 ** (-1.0 / 20)
max_audio_frame_int_value = 2 ** 15 - 1
confidence_level = 0.50

#directory to save files to
files_directory = "/Users/laurenogden/Downloads/"
models_directory = files_directory

#directories of where files are
gunshot_directory = files_directory + "all gunrange organized/gunshots/"
nongunshot_directory = files_directory + "all gunrange organized/nongunshots/"


## Functions

In [3]:
#function to save a wav file
def create_wav_file(microphone_data, name, directory = files_directory):
    librosa.output.write_wav(directory + name + ".wav", microphone_data, 22050)

In [4]:
#makes a spectrogram for 2D CNN ~~ OLD
def convert_to_spectrogram(data, sample_rate):
    return np.array(librosa.feature.melspectrogram(y = data, sr = sample_rate), dtype = "float32")

In [5]:
#functions to make a spectrogram ~~ NEW (see pipeline gunshot_detection.py)

SAMPLE_RATE_PER_SECOND = 22050
AUDIO_RATE = 44100
HOP_LENGTH = 345 * 2
MINIMUM_FREQUENCY = 20
MAXIMUM_FREQUENCY = SAMPLE_RATE_PER_SECOND
NUMBER_OF_MELS = 128
NUMBER_OF_FFTS = NUMBER_OF_MELS * 20

def convert_audio_to_spectrogram(data):
    spectrogram = librosa.feature.melspectrogram(y=data, sr=AUDIO_RATE,
                                                 hop_length=HOP_LENGTH,
                                                 fmin=MINIMUM_FREQUENCY,
                                                 fmax=MAXIMUM_FREQUENCY,
                                                 n_mels=NUMBER_OF_MELS,
                                                 n_fft=NUMBER_OF_FFTS)
    spectrogram = power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

def power_to_db(S, ref=1.0, amin=1e-10, top_db=80.0):
    S = np.asarray(S)
    if amin <= 0:
        logger.debug("ParameterError: amin must be strictly positive")
    if np.issubdtype(S.dtype, np.complexfloating):
        logger.debug("Warning: power_to_db was called on complex input so phase information will be discarded.")
        magnitude = np.abs(S)
    else:
        magnitude = S
    if six.callable(ref):
        # User supplied a function to calculate reference power
        ref_value = ref(magnitude)
    else:
        ref_value = np.abs(ref)
    log_spec = 10.0 * np.log10(np.maximum(amin, magnitude))
    log_spec -= 10.0 * np.log10(np.maximum(amin, ref_value))
    if top_db is not None:
        if top_db < 0:
            logger.debug("ParameterError: top_db must be non-negative")
        log_spec = np.maximum(log_spec, log_spec.max() - top_db)
    return log_spec

In [6]:
#auc metric for loading original models
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

## Load Models

In [7]:
#load 1D model
model_name_1D = "gunshot_sound_model.h5"
model_1D = load_model(models_directory + model_name_1D, custom_objects = {"auc":auc})

W0730 12:52:33.771265 4800861632 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 12:52:33.772101 4800861632 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 12:52:33.772786 4800861632 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensor

In [8]:
#load first 2D model
model_name_2D = "RYAN_smaller_spectrogram_model.h5"
model_2D = load_model(models_directory + model_name_2D, custom_objects = {"auc":auc})

W0730 12:52:35.886142 4800861632 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
#load second 2D model
model_name_ryan = "128_128_RYAN_smaller_spectrogram_model.h5"
model_2D_ryan = load_model(models_directory + model_name_ryan, custom_objects = {"auc":auc})

## Run Models on Data

### Create Metrics file

In [10]:
#open a file to put info about the audio clip, modes, eventually metrics, etc
metrics_filename = files_directory + "gunrange_data_model_metrics.txt"
metrics_file = open(metrics_filename, "w")
#audio file we're analyzing
metrics_file.write("Analyzing " + str(len(os.listdir(gunshot_directory)) + len(os.listdir(nongunshot_directory))) + " 2 second clips\n")
#confidence level
metrics_file.write("Predictions were done at a confidence level of " + str(confidence_level) + "\n")
metrics_file.close()

### Run Inference

#### Gunshot files first

In [11]:
#run inference on gunshot audio clips only first

true_positives_1D = []
true_positives_2D = []
true_positives_2D_ryan = []
true_positives_all = []
true_positives_all_and = []

false_negatives_1D = []
false_negatives_2D = []
false_negatives_2D_ryan = []
false_negatives_all = []
false_negatives_all_and = []


#list all of the files in the overlaid gunshot directory
#gunshot_files = os.listdir(overlaid_gunshot_directory)
gunshot_files = os.listdir(gunshot_directory)

#make predictions on each file in the directory
for file in gunshot_files:
    #make sure its a wav file
    if ".wav" in file:

        #load the file
        audio_slice, sr = librosa.load(gunshot_directory + file)

        #Normalize
        #audio_slice = normalize(audio_slice)


        #1D reshaping
        reshaped_audio_slice_1D = audio_slice.reshape(-1, 44100, 1)
        #1D predictions
        probabilities_1D = model_1D.predict(reshaped_audio_slice_1D)
        #print("1D model-predicted sample class: " + label_binarizer.inverse_transform(probabilities_1D[:, 0])[0])
        if probabilities_1D[0][1] >= confidence_level:
            predictions_1D = 1
        else:
            predictions_1D = 0

        #append either a 1 or a 0 to the predictions list based on what it predicts ~ conf level of 0.50
        #predictions_1D = np.argmax(probabilities_1D,axis=1)



        #2D reshaping
        HOP_LENGTH = 345 * 2
        reshaped_audio_slice_2D = convert_audio_to_spectrogram(data = audio_slice)
        reshaped_audio_slice_2D = reshaped_audio_slice_2D.reshape(-1, 128, 64, 1)
        #2D predictions
        probabilities_2D = model_2D.predict(reshaped_audio_slice_2D)
        if probabilities_2D[0][1] >= confidence_level:
            predictions_2D = 1
        else:
            predictions_2D = 0

        #print("2D model-predicted sample class: " + label_binarizer.inverse_transform(probabilities_2D[:, 0])[0])
        #predictions_2D.append(np.argmax(probabilities_2D,axis=1)) 



        #Ryan 2D reshaping
        HOP_LENGTH = 345
        reshaped_audio_slice_2D_ryan = convert_audio_to_spectrogram(data = audio_slice)
        #reshaped_audio_slice_2D_ryan = convert_spectrogram_to_ryan(spectrogram = reshaped_audio_slice_2D_ryan)
        reshaped_audio_slice_2D_ryan = reshaped_audio_slice_2D_ryan.reshape(-1, 128, 128, 1)
        reshaped_audio_slice_2D_ryan = reshaped_audio_slice_2D_ryan.astype("float32")
        #reshaped_audio_slice_2D_ryan /= 255
        #Ryan 2D Image predictions
        probabilities_2D_ryan = model_2D_ryan.predict(reshaped_audio_slice_2D_ryan)
        if probabilities_2D_ryan[0][1] >= confidence_level:
            predictions_2D_ryan = 1
        else:
            predictions_2D_ryan = 0

        #print("Ryan 2D model-predicted sample class: " + label_binarizer.inverse_transform(probabilities_2D_ryan[:, 0])[0])
        #predictions_2D_ryan.append(np.argmax(probabilities_2D_ryan,axis=1)) 



        #either or ensemble (if either 2D predicted gunshot, positive)
        if predictions_2D == 1 or predictions_2D_ryan ==1:
            predictions_all = 1
        else:
            predictions_all = 0

        #and ensemble (if both 2D predicted gunshot, positive)
        if predictions_2D == 1 and predictions_2D_ryan ==1:
            predictions_all_and = 1
        else:
            predictions_all_and = 0



        #based on predictions, decide if this clip was a true positive or a false negative
        if predictions_1D == 1:
            true_positives_1D.append(file)
        else:
            false_negatives_1D.append(file)

        if predictions_2D == 1:
            true_positives_2D.append(file)
        else:
            false_negatives_2D.append(file)    

        if predictions_2D_ryan == 1:
            true_positives_2D_ryan.append(file)
        else:
            false_negatives_2D_ryan.append(file)    

        if predictions_all == 1:
            true_positives_all.append(file)
        else:
            false_negatives_all.append(file)

        if predictions_all_and == 1:
            true_positives_all_and.append(file)
        else:
            false_negatives_all_and.append(file)


#### Nongunshot Files

In [12]:
# run inference on all other files (nongunshots)
true_negatives_1D = []
true_negatives_2D = []
true_negatives_2D_ryan = []
true_negatives_all = []
true_negatives_all_and = []

false_positives_1D = []
false_positives_2D = []
false_positives_2D_ryan = []
false_positives_all = []
false_positives_all_and = []


#list all of the files in the overlaid gunshot directory
#gunshot_files = os.listdir(overlaid_gunshot_directory)
nongunshot_files = os.listdir(nongunshot_directory)

#make predictions on each file in the directory
for file in nongunshot_files:
    #make sure its a wav file
    if ".wav" in file:

        #load the file
        audio_slice, sr = librosa.load(nongunshot_directory + file)

        #Normalize
        #audio_slice = normalize(audio_slice)


        #1D reshaping
        reshaped_audio_slice_1D = audio_slice.reshape(-1, 44100, 1)
        #1D predictions
        probabilities_1D = model_1D.predict(reshaped_audio_slice_1D)
        #print("1D model-predicted sample class: " + label_binarizer.inverse_transform(probabilities_1D[:, 0])[0])
        if probabilities_1D[0][1] >= confidence_level:
            predictions_1D = 1
        else:
            predictions_1D = 0

        #append either a 1 or a 0 to the predictions list based on what it predicts ~ conf level of 0.50
        #predictions_1D = np.argmax(probabilities_1D,axis=1)



        #2D reshaping
        HOP_LENGTH = 345 * 2
        reshaped_audio_slice_2D = convert_audio_to_spectrogram(data = audio_slice)
        reshaped_audio_slice_2D = reshaped_audio_slice_2D.reshape(-1, 128, 64, 1)
        #2D predictions
        probabilities_2D = model_2D.predict(reshaped_audio_slice_2D)
        if probabilities_2D[0][1] >= confidence_level:
            predictions_2D = 1
        else:
            predictions_2D = 0

        #print("2D model-predicted sample class: " + label_binarizer.inverse_transform(probabilities_2D[:, 0])[0])
        #predictions_2D.append(np.argmax(probabilities_2D,axis=1)) 



        #Ryan 2D reshaping
        HOP_LENGTH = 345
        reshaped_audio_slice_2D_ryan = convert_audio_to_spectrogram(data = audio_slice)
        #reshaped_audio_slice_2D_ryan = convert_spectrogram_to_ryan(spectrogram = reshaped_audio_slice_2D_ryan)
        reshaped_audio_slice_2D_ryan = reshaped_audio_slice_2D_ryan.reshape(-1, 128, 128, 1)
        reshaped_audio_slice_2D_ryan = reshaped_audio_slice_2D_ryan.astype("float32")
        #reshaped_audio_slice_2D_ryan /= 255
        #Ryan 2D Image predictions
        probabilities_2D_ryan = model_2D_ryan.predict(reshaped_audio_slice_2D_ryan)
        if probabilities_2D_ryan[0][1] >= confidence_level:
            predictions_2D_ryan = 1
        else:
            predictions_2D_ryan = 0

        #print("Ryan 2D model-predicted sample class: " + label_binarizer.inverse_transform(probabilities_2D_ryan[:, 0])[0])
        #predictions_2D_ryan.append(np.argmax(probabilities_2D_ryan,axis=1)) 



        #either or ensemble (if either 2D predicted gunshot, positive)
        if predictions_2D == 1 or predictions_2D_ryan ==1:
            predictions_all = 1
        else:
            predictions_all = 0

        #and ensemble (if both 2D predicted gunshot, positive)
        if predictions_2D == 1 and predictions_2D_ryan ==1:
            predictions_all_and = 1
        else:
            predictions_all_and = 0


         #based on predictions, decide if this clip was a false positive or a true negative
        if predictions_1D == 1:
            false_positives_1D.append(file)
        else:
            true_negatives_1D.append(file)

        if predictions_2D == 1:
            false_positives_2D.append(file)
        else:
            true_negatives_2D.append(file)    

        if predictions_2D_ryan == 1:
            false_positives_2D_ryan.append(file)
        else:
            true_negatives_2D_ryan.append(file)    

        if predictions_all == 1:
            false_positives_all.append(file)
        else:
            true_negatives_all.append(file)

        if predictions_all_and == 1:
            false_positives_all_and.append(file)
        else:
            true_negatives_all_and.append(file)   




## Find Metrics

In [13]:
#function to calculate metrics (precison, recall, accuracy, etc)
def find_metrics(true_positives, false_positives, false_negatives, true_negatives, model_name):
    TP = len(true_positives)
    FP = len(false_positives)
    FN = len(false_negatives)
    TN = len(true_negatives) 
    total_pos = TP + FP
    total_neg = FN + TN
    total = total_neg + total_pos


    print("------------------------------------------------------------------------------")
    print(model_name + " Model Metrics:")
    print("Total # of clips classified as gunshots: " + str(total_pos))
    print("Total # of clips classified as non-gunshots: " + str(total_neg))
    print("True Positives: " + str(TP))
    print("False Positives: " + str(FP))
    print("False Negatives: " + str(FN))
    print("True Negatives: " + str(TN))

    #calculate precision, accuracy, recall, avoid dividing by 0
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)
    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    accuracy = (TP + TN) / total
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("Accuracy: " + str(accuracy))
    print("------------------------------------------------------------------------------")
    
    #write metrics out to a text file
    metrics_file = open(metrics_filename, "a")
    metrics_file.write("------------------------------------------------------------------------------\n")
    metrics_file.write( model_name + " Model Metrics: \n")
    metrics_file.write("Total # of clips classified as gunshots: " + str(total_pos) + "\n")
    metrics_file.write("Total # of clips classified as non-gunshots: " + str(total_neg) + "\n")
    metrics_file.write("True Positives: " + str(TP) + "\n")
    metrics_file.write("False Positives: " + str(FP) + "\n")
    metrics_file.write("False Negatives: " + str(FN) + "\n")
    metrics_file.write("True Negatives: " + str(TN) + "\n")
    metrics_file.write("Precision: " + str(precision) + "\n")
    metrics_file.write("Recall: " + str(recall) + "\n")
    metrics_file.write("Accuracy: " + str(accuracy) + "\n")
    metrics_file.write("------------------------------------------------------------------------------\n")
    metrics_file.close()

### Save Metrics to txt file

In [14]:
#1D
find_metrics(true_positives_1D, false_positives_1D, false_negatives_1D, true_negatives_1D, model_name_1D)
#2D
find_metrics(true_positives_2D, false_positives_2D, false_negatives_2D, true_negatives_2D, model_name_2D)
#Ryan's 2D
find_metrics(true_positives_2D_ryan, false_positives_2D_ryan, false_negatives_2D_ryan, true_negatives_2D_ryan, model_name_ryan)
#2D ensemble - or
find_metrics(true_positives_all, false_positives_all, false_negatives_all, true_negatives_all, "2D Ensemble (or)")
#2D ensemble - and
find_metrics(true_positives_all_and, false_positives_all_and, false_negatives_all_and, true_negatives_all_and, "2D Ensemble (and)")


------------------------------------------------------------------------------
gunshot_sound_model.h5 Model Metrics:
Total # of clips classified as gunshots: 0
Total # of clips classified as non-gunshots: 2268
True Positives: 0
False Positives: 0
False Negatives: 273
True Negatives: 1995
Precision: 0
Recall: 0.0
Accuracy: 0.8796296296296297
------------------------------------------------------------------------------
------------------------------------------------------------------------------
RYAN_smaller_spectrogram_model.h5 Model Metrics:
Total # of clips classified as gunshots: 35
Total # of clips classified as non-gunshots: 2233
True Positives: 34
False Positives: 1
False Negatives: 239
True Negatives: 1994
Precision: 0.9714285714285714
Recall: 0.12454212454212454
Accuracy: 0.8941798941798942
------------------------------------------------------------------------------
------------------------------------------------------------------------------
128_128_RYAN_smaller_spectrogra

### Save clips to listen to, if you want

In [18]:
for file in false_negatives_all_and:
    audio_slice, sr = librosa.load(gunshot_directory + file)
    create_wav_file(audio_slice, "FN_" + file)

In [19]:
for file in false_positives_all:
    audio_slice, sr = librosa.load(nongunshot_directory + file)
    create_wav_file(audio_slice, "FP_" + file)

In [20]:
for file in true_positives_all:
    audio_slice, sr = librosa.load(gunshot_directory + file)
    create_wav_file(audio_slice, "TP_" + file)